# News Clusterer
This notebook will be running an incremental clustering algorithm that groups news sources into events. Based on [this research article](https://www.researchgate.net/publication/258028563_Incremental_Clustering_of_News_Reports).

In [2]:
import numpy
import nltk

## Step 1: Gathering data
We will be retrieving:

1. events that were previously clustered
1. articles that are not clustered into events
1. the global word index list

## Step 2: Preprocessing
This will stem the words down to their base forms and remove stopwords.

In [ ]:
def preprocess_string(text: str) -> str:
    pass

## Step 3: Vectorizing

This will create a vector based on the term frequency of a word. Full term weights with document frequency will be calculated in a later step to support incremental additions. 

In [ ]:
def vectorize_string(text: str) -> numpy.ndarray:
    pass

## Step 4: Clustering

This will add the vector to the cluster using the full TF-IDF weighing to compare vectors.

In [ ]:
def add_to_event(vector, events) -> list: